In [30]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

In [31]:
df = pd.read_csv('salary-train.csv')

In [32]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


Приведение к нижнему регистру и замена всего, кроме букв и цифр, на пробелы:

In [33]:
df['FullDescription'] =  df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [34]:
df['FullDescription'] = df['FullDescription'].str.lower()

In [35]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


Преобразование текста в векторы признаков с помощью TfidfVectorizer:

In [36]:
vector = TfidfVectorizer(min_df=5)

In [37]:
vec_df = vector.fit_transform(df['FullDescription'])

In [38]:
#vec_df_1 = vector.transform(df['FullDescription'])

Замена пропусков в столбцах LocationNormalized и ContractTime на специальную строку 'nan':

In [39]:
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime:

In [40]:
from sklearn.feature_extraction import DictVectorizer

In [41]:
code = DictVectorizer()

In [46]:
X = code.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объединение в матрицу объекты-признаки:

In [47]:
from scipy.sparse import hstack

In [48]:
train = hstack([vec_df, X])#по чему предсказываем
result = df['SalaryNormalized']# что предсказываем

Обучение гребневой регрессии:

In [49]:
clf = Ridge(alpha=1, random_state=241)

In [50]:
reg = clf.fit(train, result)

Тестовая выборка

In [51]:
data = pd.read_csv('salary-test-mini.csv')

In [52]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


Замена значений, приведение текста к lower, замена символов:

In [53]:
data['FullDescription'] =  data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [54]:
data['FullDescription'] = data['FullDescription'].str.lower()

In [55]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [56]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


Преобразование текста в векторы признаков с помощью TfidfVectorizer:

In [25]:
vector = TfidfVectorizer(min_df=5)
vec_df = vector.fit_transform(df['FullDescription'])
vec_data = vector.transform(data['FullDescription'])

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime:

In [57]:
X2 = code.transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объединение в матрицу объекты-признаки:

In [58]:
test = hstack([vec_data, X2])#по чему предсказываем


Прогноз:

In [59]:
res = clf.predict(test)
print(res[0], res[1])

56555.6150015 37188.3244262
